In [1]:
!git clone https://github.com/sunny442k3/KMNIST.git

Cloning into 'KMNIST'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 1), reused 20 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), 95.32 KiB | 5.02 MiB/s, done.


In [2]:
!mv ./KMNIST/* ./

In [1]:
import utils
import argparse
from trainer import Trainer
from dataset import JPDDataset
from torch.utils.data import DataLoader
from torchvision import transforms as T

In [2]:
def create_loader(cf):
    train_images, train_labels, test_images, test_labels = utils.load_dataset(cf)
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    transform = T.Compose([
        T.ToPILImage(),
        T.RandomAffine(degrees=15),
        T.RandomPerspective(),
        T.ToTensor(),
        T.Normalize(train_images.mean(), train_images.std())
    ])
    train_dataset = JPDDataset(train_images.tolist(), train_labels.tolist(), transform)
    test_dataset = JPDDataset(test_images.tolist(), test_labels.tolist())
    # return train_dataset, test_dataset
    print(f"[+] Train sample: {len(train_dataset)} - Test sample: {len(test_dataset)}")
    train_loader = DataLoader(train_dataset, batch_size=cf.batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=cf.batch_size*2, shuffle=True, pin_memory=True)
    print(f"[+] Train batch: {len(train_loader)} - Test batch: {len(test_loader)}")
    return train_loader, test_loader

In [4]:
class cf:
    dataset = "k49"
    epoch = 20
    batch_size = 4096
    learning_rate = 0.003
    checkpoint = "./checkpoint.pt"
    num_classes = 49
    input_dim = 1

In [5]:
train_loader, test_loader = create_loader(cf)

[+] Train sample: 232365 - Test sample: 38547
[+] Train batch: 57 - Test batch: 5


In [5]:
for (X, y) in train_loader:
    print(X.size(), y.size())
    break

torch.Size([512, 1, 28, 28]) torch.Size([512])


In [6]:
import torch
import torchvision
import torch.nn as nn

class ResNet(nn.Module):

    def __init__(self, cf):
        super(ResNet, self).__init__()

        model = torchvision.models.resnet101(weights=True)
        model.conv1 = torch.nn.Conv2d(cf.input_dim, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # model.fc = torch.nn.Linear(in_features=512, out_features=num_classes, bias=True)
        self.logit = model
        self.fc = nn.Linear(1000, cf.num_classes, bias=True)

    def forward(self, x):
        out = self.logit(x)
        return self.fc(out)

In [7]:
model = ResNet(cf)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=cf.learning_rate)

In [9]:
trainer = Trainer(model, optimizer, criterion)

In [ ]:
trainer.fit(train_loader, test_loader, cf.epoch, cf.checkpoint)

Running on: Tesla T4
Total update step: 1140
Epoch: 1
Train step: 10 / 57